# Целевая функция

Нижний индекс - номер числка/вектора в наборе.

$$ x, s, a_i \in \mathbb{R}_+^n;\quad c \in \mathbb{R}_+^m; \quad i=\overline{1,m} $$

$$ f(x) = s^T x - \sum_{i=1}^m c_i \log(a_i^T x) + h(x) $$

$$ \nabla f(x) = s + \sum_{i=1}^m \dfrac{c_i}{a_i^T x}a_i + \nabla h(x) $$

In [3]:
import numpy as np

def f(x, s, c, a, h):
    # h - penalty function
    # s,c,a - const (see problem statement)
    return np.dot(s, x) + sum(c[:] * np.log(np.dot(a[:][:], x))) + h(x)

def grad_f(x, s, c, a, grad_h):
    # grad_h - penalty function gradient
    # s,c,a - const (see problem statement)
    return s + sum(c[:] / np.dot(a[:], x) * a[:]) + grad_h(x)

# Задача

$$ \min_{x\in\mathbb{R}^n_+} s^tx - \sum_{i=1}^m  c_i \log(a_i^T x) + h(x) $$

Или, переформулируя как задачу о седловой точке, 

$$ \min_{x\in\mathbb{R}^n_+} \max_{y\in\mathbb{R}^m_{++}}  s^tx - y^TAx + \sum_{i=1}^m  c_i \log(y_i) + h(x) + c_0 $$

где $A = [a_1^T, \ldots, a_m^T]$ и $c_0 = \sum_{i=1}^m c_i\left( 1 - \log c_i \right)$ - константа.

# Градиентный спуск

В общем виде алгоритм градиентного спуска (верхний индекс - номер итерации):

$$ x^{k+1} = x^{k} - \gamma^{k}\nabla f(x^k) $$

При это алгоритм вычисления значения фунции и градиента функции в точке известен и имеет трудоёмкость $O(m)$.

In [22]:
class GradientDescend:
    
    dim   = 0        # dimension
    grad  = None     # objective function gradient
    alpha = None     # MD coef
    coef_func = None # gamma^k

    def default_coef_func(it_num, x, grad_val):
        return 0.01
    
    def __init__(self, _dim, _grad, _alpha):
        self.dim   = _dim
        self.alpha = _alpha
        self.grad  = _grad
    
    def do_nothing(x, t):
        return
    
    def run(self, x0, steps_num = 1000, user_action = do_nothing):
        x      = np.zeros(self.dim)
        next_x = np.zeros(self.dim)
        
        x = x0
        user_action(x, 0)

        for i in range(1, steps_num):
            next_x = self.iteration(i, x)
            user_action(next_x, i)
            x, next_x = next_x, x # swap array references

        return x
    
    def iteration(self, iter_num, x):
            return x - np.dot(self.alpha(iter_num, x, self.grad(x)), self.grad(x))

## Пример использования

In [23]:
h_test      = lambda x : 0.0
grad_h_test = lambda x : np.zeros(len(x))
alpha_test  = lambda k, x, grad_f: 0.001

df = lambda x: 4 * x**3 - 9 * x**2 # f = x^4 - 3x^3

x0 = [1.0, 2.0, 3.0]
s  = [1.0, 0.0, 0.0]
c  = [1.0, -1.0, 0.0]
a  = [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]

md = GradientDescend(len(x0), df, alpha_test)
print "GD : y* =", md.run(1.0, 10000), " y =", 2.25

grad_test = lambda x: grad_f(x, s, c, a, grad_h_test)

md = GradientDescend(len(x0), grad_test, alpha_test)
md.run(x0, 100)

GD : y* = 2.2499999999999893  y = 2.25


array([0.81586945, 2.04677104, 3.        ])

# Зеркальный спуск

В общем виде алгоритм зеркального спуска (верхний индекс - номер итерации):

$$ x^{k+1} = \arg \min_{x \in \mathbb{R}_+^n} ~ \left\langle \alpha^k \nabla f(x^k), x \right\rangle + V_{x^k}(x) $$

Где градиент исследуемой функции $f(x)$:

$$ \nabla f(x^k) = \alpha^k\left(s + \sum_{i=1}^m \dfrac{c_i}{a_i^T x^k}a_i + \nabla h(x^k)\right)$$

В нашей задачи для оценки сходимости используется первая норма, поэтому в качестве расстояния Брэгмана мы выберем 

$$ V_{x}(y) = \sum_{i=1}^n x_i \log \dfrac{y_i}{x_i}$$

Итого, переходя к нормированным переменным $\tilde{x}\in\Delta_n$ (после переходя со звёздочкой волну над $x$ опустим):

$$ x^{k+1} = \arg \min_{x \in \mathbb{R}_+^n}  \left\langle \alpha^k \nabla f(x^k), x \right\rangle + \sum_{i=1}^n x^k_i \log \dfrac{x_i}{x^k_i} =^* $$

$$ =^*  \arg \min_{x \in \Delta_n} \left\langle \alpha^k \nabla f(x^k), x \right\rangle + \sum_{i=1}^n x^k_i \log \dfrac{x_i}{x^k_i} = $$

$$ = x^k \cdot \dfrac{\exp\left(-\alpha^k \nabla f(x^k) \right)}{\left\lVert x^k\cdot \exp\left(-\alpha^k \nabla f(x^k)\right) \right\rVert_1}$$

In [26]:
class MirrorDescend:
    
    dim   = 0     # dimension
    grad  = None  # objective function gradient
    alpha = None  # MD coef

    def __init__(self, _dim, _grad, _alpha):
        self.dim   = _dim
        self.alpha = _alpha
        self.grad  = _grad
    
    def do_nothing(x, t):
        return
    
    def run(self, x0, steps_num = 1000, user_action = do_nothing):
        x      = np.zeros(self.dim)
        next_x = np.zeros(self.dim)
        
        x = x0
        user_action(x, 0)

        for i in range(1, steps_num):
            next_x = self.iteration(i, x)
            user_action(next_x, i)
            x, next_x = next_x, x # swap array references

        return x
    
    def iteration(self, iter_num, x):
            z = np.exp(- self.alpha(iter_num) * self.grad(x))
            return x * z / np.linalg.norm(x * z, 1)

## Пример использования

In [27]:
h_test      = lambda x : 0.0
grad_h_test = lambda x : np.zeros(len(x))
alpha_test  = lambda k : 0.001

x0 = [1.0, 2.0, 3.0]
s  = [1.0, 0.0, 0.0]
c  = [1.0, -1.0, 0.0]
a  = [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]

grad_test   = lambda x : grad_f(x, s, c, a, grad_h_test)

md = MirrorDescend(len(x0), grad_test, alpha_test)
md.run(x0, 100)

array([0.08914119, 0.42557429, 0.48528452])

# Composite mirror prox algorithm

Рассмотрим задачу задачу

$$ \min_{u_1\in U_1} \max_{u_1\in U_1} \phi(u_1,u_2) + \Psi_1(u_1) - \Psi_2(u_2)$$

Для неё известен алгоритм "Composite Mirror Prox".

Pаметим теперь, что данная задача является эквивалентной нашей. Действительно, 

$$ \min_{u_1\in U_1} \max_{u_1\in U_1} \phi(u_1,u_2) + \Psi_1(u_1) - \Psi_2(u_2) ~=~ \min_{x\in\mathbb{R}^n_+} \max_{y\in\mathbb{R}^m_{++}}  s^Tx - y^TAx + \sum_{i=1}^m  c_i \log(y_i) + h(x) + c_0 $$

$$u_1 = x, ~ U_1 = \mathbb{R}^n_+\\ u_2 = y, ~U_2 = \mathbb{R}^m_{++}$$

$$\phi(x,y) + \Psi_1(x) - \Psi_2(y) ~=~ s^Tx - y^TAx + \sum_{i=1}^m  c_i \log(y_i) + h(x) + c_0 $$

$$ \phi(x,y) =  s^tx - y^TAx + c_0$$
$$ \Psi_1(x) = h(x);  \quad \Psi_2(y) = - \sum_{i=1}^m  c_i \log(y_i)  $$

Таким образом алгоритм может быть применён для решения поставленной задачи.

In [30]:
from scipy.optimize import minimize

phi      = lambda x, y, s, A, c : np.dot(s, x) - np.dot(np.dot(y, A), x) + c
grad_phi = lambda x, y, s, A, c : s - np.dot(A, y)

psi_1 = lambda x, h : h(x)
psi_2 = lambda y, c : sum(c[:] * np.log(np.dot(a[:][:], y)))


class CompositeMirrorProx:
    
    dim    = 0     # dimension
    func   = None
    grad_1 = None
    grad_2 = None
    psi_1  = None
    psi_2  = None
    breg1  = None
    breg2  = None
    alpha1 = None
    alpha2 = None

    gamma = None  # MD coef

    def __init__(self, _dim, _func, _grad_1, _grad_2, _psi1, _psi2, 
                 _alpha1, _alpha_2, _breg1, _breg2, _gamma):
        self.dim    = _dim
        self.func   = _func
        self.grad_1   = _grad_1
        self.grad_2   = _grad_2
        self.psi_1  = _psi1
        self.psi_2  = _psi2
        self.breg1  = _breg1
        self.breg2  = _breg2
        self.alpha1 = _alpha1
        self.alpha2 = _alpha2
        self.gamma  = _gamma

    def do_nothing(x, t):
        return
    
    def run(self, u0_1, u0_2, steps_num = 1000, user_action = do_nothing):
        u      = np.zeros(self.dim)
        next_u = np.zeros(self.dim)
        
        scipy.optimize.LinearConstraint 
        
        u_1 = u0_1
        u_2 = u0_2
        user_action([u_1, u_2], 0)    
        
        for i in range(1, steps_num):
            next_u_1, next_u_2 = self.iteration(t, t, u0_1, u0_2, u_1, u_2)
            user_action([next_u_1, next_u_2], t)
            u_1, next_u_1 = next_u_1, u_1 # swap array references
            u_2, next_u_2 = next_u_2, u_2 # swap array references

        return x
    
    
    def opt_func_1(t, u_1, u_t_1, u_cap1, u_cap2):
        return self.alpha1 * self.Breg1(u_1, u_t_1) + \
               np.dot(self.gamma(t) * self.grad_1(u_cap1, u_cap2), u_1) + \
               self.gamma(t) * self.psi_1(u_1)

    def opt_func_2(t, u_2, u_t_2, u_cap1, u_cap2):
        return self.alpha2 * self.Breg2(u_2, u_t_2) + \
               np.dot(self.gamma(t) * self.grad_2(u_cap1, u_cap2), u_2) + \
               self.gamma(t) * self.psi_2(u_2)
    
    def opt(func, t, u, u_t, u_cap1, u_cap2):
        return minimize(func(t, u, u_t, u_cap1, u_cap2), method='nelder-mead', bounds=(0,np.inf))

    def iteration(self, t, u0_1, u0_2, u_1, u_2):       
            u_cap_1   = self.opt(self.opt_func_1, t, u0_1, u_1, u_1, u_2)
            u_cap_2   = self.opt(self.opt_func_2, t, u0_2, u_2, u_1, u_2)
            next_u_1  = self.opt(self.opt_func_1, t, u0_1, u_1, u_cap_1, u_cap_2)
            next_u_2  = self.opt(self.opt_func_2, t, u0_2, u_2, u_cap_1, u_cap_2)
            return next_u_1, next_u_2